In [34]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , mean_absolute_error
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
import joblib

### Load model :

In [2]:
model=load_model('/home/fayari/Stage/modeling/deep_model_v2.h5', compile=False)

2025-06-18 11:03:17.771750: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Load Data :

In [38]:
duplicated_data=pd.read_csv('/home/fayari/Stage/data_exploration_2/noisy_duplicated_data.csv')
df_no_outliers=pd.read_csv('/home/fayari/Stage/data_exploration/no_outliers.csv')
df_no_outliers_SDB = pd.read_csv('/home/fayari/Stage/data_exploration_2/no_outliers.csv')
df_no_outliers_2_SDB = pd.read_csv('/home/fayari/Stage/data_exploration_2/no_outliers_2.csv')
augmented_data=pd.read_csv('/home/fayari/Stage/data_augmentation/add_training.csv')
#df_no_outliers.drop(columns=['BOA_RT'],inplace=True)
#new = pd.read_csv('/home/fayari/Stage/modeling_BOA/add_training_v2.csv')
data=pd.concat([duplicated_data,df_no_outliers,df_no_outliers_SDB,df_no_outliers_2_SDB,augmented_data], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
#data.drop(columns=['BOA_RT'],inplace=True)

In [39]:
scaler=joblib.load('/home/fayari/Stage/modeling/scaler_SS_DART.pkl')
scaler_y=joblib.load('/home/fayari/Stage/modeling/scaler_y_SS_DART.pkl')

In [40]:
cols_to_scale = data.drop(columns=['alpha','BOA_RT']).columns

In [41]:
X_scaled = scaler.transform(data[cols_to_scale])
y_scaled=scaler_y.transform(data['alpha'].values.reshape(-1, 1))
BOA=data['BOA_RT']

In [42]:
X_train,X_val,y_train,y_val,BOA_train,BOA_val=train_test_split(X_scaled,y_scaled,BOA,test_size=0.2,random_state=1042)

In [49]:
y_pred=model.predict(X_val)

23307/23307 ━━━━━━━━━━━━━━━━━━━━ 55s 2ms/step


In [50]:
y_pred

array([[-0.15279764],
       [-7.7379293 ],
       [ 0.54727197],
       ...,
       [ 0.3107924 ],
       [ 0.42929542],
       [-0.49273062]], dtype=float32)

In [51]:
X_scaled

array([[ 0.86483348, -0.41618962,  0.45093405, ..., -0.6765415 ,
        -0.08585945, -0.10592483],
       [-0.9773016 ,  0.47277182,  0.19672657, ...,  1.28283715,
        -0.57520843, -0.51346391],
       [-3.0924652 ,  0.42907262,  0.11596268, ..., -1.38325555,
         0.78726986,  0.25710077],
       ...,
       [-1.39923671,  0.47649146,  0.17708131, ...,  1.35330473,
        -0.54876767, -0.42651308],
       [ 0.59188767,  0.24652511,  0.32114663, ..., -0.17250546,
        -0.2694443 , -0.13762987],
       [-0.81865977,  0.47510255,  0.20545781, ..., -1.38325523,
         0.90860553,  0.60233029]])

In [ ]:
X_train = pd.DataFrame(X_train, columns=data[cols_to_scale].columns)
X_val = pd.DataFrame(X_val, columns=data[cols_to_scale].columns)
X_train

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.749828,0.490601,0.270942,0.102553,-0.619645,-0.773631,-0.767586,1.495556,0.983251,-1.013819,0.872718,-1.777017,0.983251,0.983251,-1.747185,-1.768129,4.334749,2.615751
1,0.335043,0.125146,0.288405,0.398303,-0.327624,-0.051468,0.041153,0.680533,0.532227,0.886317,-0.141683,-0.622994,0.532227,0.532227,-0.627552,-0.624227,-0.159803,-0.253743
2,0.088832,0.309225,0.441635,0.420416,-0.146143,-0.795646,-0.739464,1.417934,1.415840,0.332219,-0.570909,-1.663970,1.415840,1.415840,-1.651024,-1.660926,2.274568,0.835677
3,-1.084889,0.474393,0.192361,0.436740,0.819164,0.711416,0.860025,-0.561406,-0.323121,-0.642834,-0.171696,0.766449,-0.323121,-0.323121,0.766701,0.766543,-0.486033,-0.379992
4,-0.471528,0.461835,0.225103,0.428926,0.292318,0.466592,0.597844,1.340313,1.780838,-0.974933,-0.327600,-1.551199,1.780838,1.780838,-1.546106,-1.550269,1.300775,0.221760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983256,0.860952,-2.723330,0.531019,-2.714329,-0.695156,-1.764596,-1.789942,-0.062084,-0.093712,-0.741048,0.289334,0.285551,-0.093712,-0.093712,0.283698,0.285087,-0.417771,-0.350457
2983257,0.072831,0.303266,0.266576,0.420021,-0.134125,0.177702,0.289330,-0.483785,1.720619,-1.152770,-0.561455,0.699910,1.720619,1.720619,0.699857,0.699924,-0.537541,-0.525817
2983258,0.555487,0.407181,0.318964,0.368389,-0.484875,-0.329562,-0.258757,1.223881,-0.721300,0.822909,0.828354,-1.382859,-0.721300,-0.721300,-1.383259,-1.383255,1.011518,0.969022
2983259,0.576554,-0.271176,0.323329,0.365060,-0.499655,-0.362667,-0.294236,1.029828,-0.525898,1.828772,0.964766,-1.105608,-0.525898,-0.525898,-1.109355,-1.106740,0.356673,0.323081


In [52]:
y_pred_val_real=scaler_y.inverse_transform(y_pred)

In [53]:
y_pred_val_real

array([[ 0.00548002],
       [-4.779985  ],
       [ 0.44715443],
       ...,
       [ 0.2979593 ],
       [ 0.37272295],
       [-0.20898393]], dtype=float32)

In [54]:
y_val_real=scaler_y.inverse_transform(y_val)

In [57]:
X_val_inverse = scaler.inverse_transform(X_val)

# Recréer un DataFrame avec les noms de colonnes d'origine
X_val_real = pd.DataFrame(X_val_inverse, columns=data[cols_to_scale].columns)
X_val_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.973800,0.997500,0.984600,0.927161,0.026549,0.213069,0.197550,47.0,614.0,0.661646,0.949688,0.681998,707.957111,3185.8070,0.682551,0.682121,1.368473,1.078475
1,0.988300,0.627393,0.997206,0.896484,0.011769,0.027032,0.024234,52.0,768.0,0.789646,0.945366,0.615661,707.974222,3185.8840,0.616372,0.615820,1.489695,1.106057
2,0.921262,0.942704,0.984260,0.948516,0.082010,0.308168,0.292302,75.0,0.0,0.150000,0.887287,0.258819,707.888889,3185.5000,0.261337,0.259383,3.826474,3.855296
3,0.948600,0.968475,0.985300,0.944879,0.052768,0.268664,0.253855,61.0,648.0,0.464156,0.923353,0.484810,707.960889,3185.8240,0.485920,0.485057,1.914988,1.491062
4,0.928000,0.955827,0.984500,0.947816,0.074724,0.299350,0.283728,2.0,658.0,0.254493,0.894605,0.999391,707.962000,3185.8290,0.999392,0.999391,0.930063,0.720082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745811,0.654600,0.996601,0.976519,0.953386,0.423731,0.509729,0.485968,63.0,780.0,0.590092,0.901972,0.453990,707.975556,3185.8900,0.455220,0.454265,2.014371,1.490446
745812,0.957911,0.985487,0.985800,0.943258,0.043000,0.252048,0.237746,43.0,120.0,0.440000,0.941489,0.731354,707.902222,3185.5600,0.731802,0.731453,1.348391,1.287525
745813,0.813700,0.995556,0.981300,0.951498,0.206164,0.389198,0.370321,85.0,1822.0,0.449733,0.889850,0.087156,708.091333,3186.4110,0.094557,0.088907,8.637483,4.522963
745814,0.958700,0.986500,0.985800,0.942910,0.042177,0.250515,0.236213,80.0,739.0,0.019777,0.943256,0.173648,707.971000,3185.8695,0.177502,0.174520,5.189641,3.959887


In [58]:
X_val_real['alpha']=y_val_real
X_val_real['alpha_pred']=y_pred_val_real

In [59]:
X_val_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,alpha,alpha_pred
0,0.973800,0.997500,0.984600,0.927161,0.026549,0.213069,0.197550,47.0,614.0,0.661646,0.949688,0.681998,707.957111,3185.8070,0.682551,0.682121,1.368473,1.078475,-0.009872,0.005480
1,0.988300,0.627393,0.997206,0.896484,0.011769,0.027032,0.024234,52.0,768.0,0.789646,0.945366,0.615661,707.974222,3185.8840,0.616372,0.615820,1.489695,1.106057,-4.949676,-4.779985
2,0.921262,0.942704,0.984260,0.948516,0.082010,0.308168,0.292302,75.0,0.0,0.150000,0.887287,0.258819,707.888889,3185.5000,0.261337,0.259383,3.826474,3.855296,0.452658,0.447154
3,0.948600,0.968475,0.985300,0.944879,0.052768,0.268664,0.253855,61.0,648.0,0.464156,0.923353,0.484810,707.960889,3185.8240,0.485920,0.485057,1.914988,1.491062,0.189277,0.193264
4,0.928000,0.955827,0.984500,0.947816,0.074724,0.299350,0.283728,2.0,658.0,0.254493,0.894605,0.999391,707.962000,3185.8290,0.999392,0.999391,0.930063,0.720082,0.161574,0.172549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745811,0.654600,0.996601,0.976519,0.953386,0.423731,0.509729,0.485968,63.0,780.0,0.590092,0.901972,0.453990,707.975556,3185.8900,0.455220,0.454265,2.014371,1.490446,0.243175,0.255123
745812,0.957911,0.985487,0.985800,0.943258,0.043000,0.252048,0.237746,43.0,120.0,0.440000,0.941489,0.731354,707.902222,3185.5600,0.731802,0.731453,1.348391,1.287525,0.231750,0.217078
745813,0.813700,0.995556,0.981300,0.951498,0.206164,0.389198,0.370321,85.0,1822.0,0.449733,0.889850,0.087156,708.091333,3186.4110,0.094557,0.088907,8.637483,4.522963,0.292950,0.297959
745814,0.958700,0.986500,0.985800,0.942910,0.042177,0.250515,0.236213,80.0,739.0,0.019777,0.943256,0.173648,707.971000,3185.8695,0.177502,0.174520,5.189641,3.959887,0.367199,0.372723


In [30]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    
    BOA_ia= numerator / denominator
    return BOA_ia


In [60]:
X_val_real['BOA_pred'] = X_val_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha_pred'],row['GOD'],row['AODS']), axis=1)

In [61]:
X_val_real['BOA_RT']=BOA_val.values

In [62]:
X_val_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,alpha,alpha_pred,BOA_pred,BOA_RT
0,0.973800,0.997500,0.984600,0.927161,0.026549,0.213069,0.197550,47.0,614.0,0.661646,...,707.957111,3185.8070,0.682551,0.682121,1.368473,1.078475,-0.009872,0.005480,979.462532,981.078389
1,0.988300,0.627393,0.997206,0.896484,0.011769,0.027032,0.024234,52.0,768.0,0.789646,...,707.974222,3185.8840,0.616372,0.615820,1.489695,1.106057,-4.949676,-4.779985,569.895002,572.840359
2,0.921262,0.942704,0.984260,0.948516,0.082010,0.308168,0.292302,75.0,0.0,0.150000,...,707.888889,3185.5000,0.261337,0.259383,3.826474,3.855296,0.452658,0.447154,573.693784,572.034744
3,0.948600,0.968475,0.985300,0.944879,0.052768,0.268664,0.253855,61.0,648.0,0.464156,...,707.960889,3185.8240,0.485920,0.485057,1.914988,1.491062,0.189277,0.193264,881.261790,882.027225
4,0.928000,0.955827,0.984500,0.947816,0.074724,0.299350,0.283728,2.0,658.0,0.254493,...,707.962000,3185.8290,0.999392,0.999391,0.930063,0.720082,0.161574,0.172549,926.137197,927.505307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745811,0.654600,0.996601,0.976519,0.953386,0.423731,0.509729,0.485968,63.0,780.0,0.590092,...,707.975556,3185.8900,0.455220,0.454265,2.014371,1.490446,0.243175,0.255123,749.288741,757.030657
745812,0.957911,0.985487,0.985800,0.943258,0.043000,0.252048,0.237746,43.0,120.0,0.440000,...,707.902222,3185.5600,0.731802,0.731453,1.348391,1.287525,0.231750,0.217078,930.277816,928.171791
745813,0.813700,0.995556,0.981300,0.951498,0.206164,0.389198,0.370321,85.0,1822.0,0.449733,...,708.091333,3186.4110,0.094557,0.088907,8.637483,4.522963,0.292950,0.297959,520.650297,524.270558
745814,0.958700,0.986500,0.985800,0.942910,0.042177,0.250515,0.236213,80.0,739.0,0.019777,...,707.971000,3185.8695,0.177502,0.174520,5.189641,3.959887,0.367199,0.372723,735.170150,736.973636


In [63]:
MAE=mean_absolute_error(X_val_real['BOA_RT'],X_val_real['BOA_pred'])
MAE

4.410450259045047